In [1]:
import pandas as pd
import torch

In [2]:
pd.set_option('display.max_columns',None)
df = pd.read_csv('final_cohort_variable1_6_shix.csv')

In [3]:
for column in df.columns[df.isnull().sum()>len(df)/2]:
    df.drop([column],axis=1,inplace=True)

In [79]:
print(df['weight'])

0         86.0
1         86.0
2         78.0
3         78.0
4         78.0
          ... 
318895    86.0
318896    86.0
318897    86.0
318898    86.0
318899    90.0
Name: weight, Length: 183464, dtype: float64


In [75]:
df.describe()

,subject_id,hadm_id,stay_id,age,gender,weight,readmission,hospital_expire_flag,sirs,charlson,heart_rate,sbp,mbp,dbp,temperature,spo2,gcs,is_ventilation,urine
count,1.834640e+05,1.834640e+05,1.834640e+05,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000,183464.000000
mean,1.498804e+07,2.498282e+07,3.447505e+07,65.236406,0.559903,81.627287,0.242816,0.106190,2.727363,5.893336,84.553868,118.869591,78.249720,62.846143,36.917822,96.855566,14.515098,0.075688,350.189133
std,2.886890e+06,2.885762e+06,2.892803e+06,17.027097,0.496400,45.426953,0.428786,0.308081,0.936270,3.232204,16.934931,18.038499,12.540471,12.432201,0.600068,2.445178,1.389474,0.264499,304.617894
min,1.000122e+07,2.000009e+07,3.000015e+07,18.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,27.750000,39.500000,14.000000,14.750000,29.900000,9.000000,3.000000,0.000000,-5850.000000
25%,1.248812e+07,2.248675e+07,3.192143e+07,55.000000,0.000000,65.900000,0.000000,0.000000,2.000000,4.000000,72.250000,105.750000,69.400000,54.000000,36.600000,95.500000,15.000000,0.000000,150.000000
50%,1.498919e+07,2.496327e+07,3.420324e+07,67.000000,1.000000,78.200000,0.000000,0.000000,3.000000,6.000000,83.000000,116.750000,76.750000,61.500000,36.890000,97.000000,15.000000,0.000000,260.000000
75%,1.749902e+07,2.747596e+07,3.690966e+07,78.000000,1.000000,93.000000,0.000000,0.000000,3.000000,8.000000,95.250000,130.000000,85.670000,70.250000,37.220000,98.750000,15.000000,0.000000,450.000000
max,1.999999e+07,2.999983e+07,3.999981e+07,99.000000,1.000000,6492.790000,1.000000,1.000000,4.000000,20.000000,177.110000,228.000000,202.250000,161.670000,41.330000,100.000000,15.000000,1.000000,5550.000000


In [5]:
miss = df.isnull().sum(axis=1)
df.drop(index=miss[miss>0].index,inplace=True)

In [6]:
mis = df['subject_id'].value_counts()

In [7]:
import numpy as np
subject_id = df['subject_id'].value_counts().index
num = np.array(df['subject_id'].value_counts())
count = pd.DataFrame(np.vstack([subject_id,num]).T,columns=['subject_id','count'])
df2 = pd.merge(df,count,how='inner',on='subject_id')
mis = df2['count']<6
df3 = df2.drop(index=mis[mis].index)

In [8]:
df3.index = range(len(df3))
X = df3.copy()
X = X.drop_duplicates(['subject_id'])
X = X.iloc[:,[4,5,6,7,8,9,10]]
df_died = X['hospital_expire_flag']#用来做决策任务

In [9]:
# from sklearn.cluster import KMeans
# from sklearn import metrics
# kmeans_model = KMeans(n_clusters=5, random_state=10).fit(X)
# labels = kmeans_model.labels_

In [10]:
#X['label'] = labels

In [11]:
#df3['label']=df_died
# for i in X.index:
#     for j in range(6):
#         df3.loc[i+j,'label']=X.loc[i,'label']

确认用什么标准化很关键

In [12]:
data = df3.iloc[:,range(11,20)]
data = (data-data.min())/(data.max()-data.min())
#data = (data - data.mean())/data.std()

In [13]:
#Label = df3.iloc[:,21]

In [14]:
Label = df_died

In [15]:
import torch
import torch.nn
data_iter = []
for i in range(0,len(data),6):
    data_iter.append(np.array(data.iloc[i:i+6,:]).tolist())

In [16]:
data_iter = torch.Tensor(data_iter)
data_iter = data_iter.unsqueeze(1)
data_iter.shape

torch.Size([9926, 1, 6, 9])

In [17]:
data_iter_train = data_iter[:7000]
data_iter_test  = data_iter[9626:9926]

In [18]:
Label = torch.Tensor(Label)
print(Label.shape)

torch.Size([9926])


In [19]:
Label1 = Label[:7000]
Label2 = Label[9626:9926]

In [20]:
import torch.utils.data as Data
torch_dataset = Data.TensorDataset(data_iter_train,Label1)
loader = Data.DataLoader(dataset = torch_dataset,batch_size=128,shuffle=True)

VAE－LSTM－GAIN

1.matrix initial

In [109]:
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

missing_rate = 0.6#设置随机缺失率

def matrix_init(mat):
    """
    获取mask矩阵binary_matrix，随机缺失seed矩阵random_matrix,和随机缺失矩阵sparse_matrix
    """
    high = mat.shape[0]
    row = mat.shape[1]
    col = mat.shape[2]
    
    binary_matrix = np.zeros((high, row, col))
    random_matrix = np.random.random((high, row, col))
    for h in range(high):
        for i in range(row):
            for j in range(col):
                binary_matrix[h, i, j] = np.round(random_matrix[h, i,j] + 0.5 - missing_rate)
    
    sparse_matrix = np.multiply(mat, binary_matrix)
    return torch.tensor(binary_matrix), sparse_matrix

2.Neural Network decomposition

initial filling based on time regular data(up to down)

SVD decomposition                 

filling NaN in sparse_matrix(multiple with binary_matrix and don't change the orginal data


get the first-repaired matrix

this is the input in VAE

In [22]:
import torch
import numpy as np
import pandas as pd

def means_filling(mat):
    """
    先用均值进行填充
    pay attention to the data type in np.array
    """
    for h in range(mat.shape[0]):
        means = mat.sum(axis=1)/((mat>0).sum(axis=1)+1)
        for i in range(mat.shape[1]):
            for j in range(mat.shape[2]):
                if mat[h, i, j] == 0:
                    mat[h, i, j] = means[h, j]
    return torch.tensor(mat)

In [23]:
from scipy import linalg
def SVD_func(full_matrix, binary_matrix, sparse_matrix):
    """
    返还初次填补矩阵
    """
    mat = np.zeros((full_matrix.shape[0], full_matrix.shape[1], full_matrix.shape[2]))
    for h in range(full_matrix.shape[0]):
        u, s, k = linalg.svd(full_matrix[h].reshape((full_matrix.shape[1],full_matrix.shape[2])))
        mat[h,:s.shape[0], :s.shape[0]] = np.diag(s)
        mat[h] = np.matmul(u, mat[h])
        mat[h] = np.matmul(mat[h], k)
    binary_matrix_reverse = torch.tensor(np.ones((binary_matrix.shape[0], binary_matrix.shape[1], binary_matrix.shape[2])))-binary_matrix
    return np.multiply(mat, binary_matrix_reverse) + sparse_matrix

3.VAE-GAN

VAE structure

Time Convolution - Generator

In [24]:
import os
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, sampler
from torchvision.datasets import MNIST
from torchvision import transforms as tfs
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

#VAE结构如下，也是生成器

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(-1, 6, 9)

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.f1 = Reshape()
        self.fc1 = nn.Conv1d(1, 1, kernel_size = (4, 1))
        self.fc2 = nn.Conv1d(1, 1, kernel_size = (3, 1))
        self.fc21 = nn.Linear(9, 9) # mean
        self.fc22 = nn.Linear(9, 9) # var
        self.fc3 = nn.ConvTranspose1d(1, 1, kernel_size = (3, 1))
        self.fc4 = nn.ConvTranspose1d(1, 1, kernel_size = (4, 1))

    def encode(self, x): #编码层
        h1 = torch.relu(self.fc1(x))
        h1 = torch.relu(self.fc2(h1))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_() #e**(x/2)
        eps = torch.FloatTensor(std.size()).normal_()
        if torch.cuda.is_available():
            eps = Variable(eps.cuda())
        else:
            eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):#解码层
        h3 = torch.relu(self.fc3(z))
        return torch.tanh(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x) # 编码
        z = self.reparametrize(mu, logvar) # 重新参数化成正态分布
        return self.decode(z), mu, logvar # 解码，同时输出均值方差

LSTM - Discriminator

In [25]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
            #Input:9926*1*6*9
#         self.lstm = nn.LSTM(#类比每次输入1个字，6个时间点，每个字是9个维度，输出为隐藏层数目27
#                 9,
#                 27,
#                 2, #层数为2
#                 batch_first=True
#             )#4*4
#         self.l1 = nn.Linear(162, 128)
#         self.l3 = nn.Linear(128, 54)
#         self.l2 = Reshape()
#         self.l4 = nn.Dropout(p=0.2)
# #         可以考虑用softmax
#         for name, param in self.lstm.named_parameters():
#             nn.init.uniform_(param,0,0.1)

        self.l1 = nn.Conv1d(1, 1, kernel_size = (4, 1))
        self.l2 = nn.Conv1d(1, 1, kernel_size = (3, 1))
        self.l3 = nn.Linear(9 , 27)
        self.l4 = nn.Linear(27, 54)
        self.l5 = Reshape()
    
    def forward(self,x):
#         out, _ = self.lstm(x)
        #out = out[:, -1, :] #可以考虑对参数进行分类
#         out = torch.sigmoid(out)
#         out = nn.Flatten(out)
#         out = out.reshape(-1, 162)
#         ln1 = self.l1(out)
#         ln2 = self.l3(ln1)
#         ln2 = torch.sigmoid(ln2)
#         ln2 = self.l2(ln2)
#         return ln2
        x = x.view(-1,1,6,9)
        out = self.l1(x)
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        out = torch.sigmoid(out)
        return self.l5(out)

Loss function

$$\hat{X}=M\circledast\tilde{X}+(1-M)\circledast G\left(\tilde{X},M\right)$$

In [26]:
def ls_discriminator_loss(discirminator_matrix, binary_matirx): #判别器的loss
#     print(binary_matirx.shape)
#     binary_matirx = binary_matirx.reshape(1,binary_matirx.shape[1],binary_matirx.shape[2])
#     binary_matirx = binary_matirx.expand(1000,binary_matirx.shape[1],binary_matirx.shape[2])
    loss = -sum(sum(sum(binary_matirx * torch.log(discirminator_matrix + 1e-8) + (1 - binary_matirx) * torch.log(1. - discirminator_matrix + 1e-8))))
    return loss/(binary_matirx.shape[0] * binary_matirx.shape[1] * binary_matirx.shape[2])

def MSE_train_loss(binary_matrix, origin_matrix, G_sample):
    loss = sum(sum(sum(( binary_matrix * origin_matrix - binary_matrix * G_sample) ** 2)))/sum(sum(sum(binary_matrix)))
    return loss

def ls_generator_loss(discirminator_matrix, binary_matirx):
#     binary_matirx = binary_matirx.reshape(1,binary_matirx.shape[1],binary_matirx.shape[2])
#     binary_matirx = binary_matirx.expand(1000,binary_matirx.shape[1],binary_matirx.shape[2])
    loss = -sum(sum(sum((1 - binary_matirx) * torch.log(discirminator_matrix + 1e-8))))
    return loss/(binary_matirx.shape[0] * binary_matirx.shape[1] * binary_matirx.shape[2])
    

In [27]:
#先reshape将二维张量转化为三维张量，然后再将三维张量平移扩展expand达到赋值二维张量到三维的操作

Adam optimizer

In [28]:
def get_optimizer(net):
    optimizer = torch.optim.Adam(net.parameters(), lr=3e-3, betas=(0.5, 0.999))
    return optimizer

MSE_train

In [29]:
def MSE_train(binary_matrix, origin_matrix, G_sample):
    loss = sum(sum(sum(((1 - binary_matrix) * origin_matrix - (1 - binary_matrix) * G_sample) ** 2)))/sum(sum(sum(1 - binary_matrix)))
    return loss

train

In [30]:
alpha = 1
def train_a_gan(D_net, G_net, D_optimizer, G_optimizer, discriminator_loss, generator_loss, show_every=9926, num_epochs=100):
    iter_count = 0
    for epoch in range(num_epochs):
        for x, (real_data,_) in enumerate(loader):
            #real_data = x.reshape(-1,6,9)
            #bs = x.shape[0]   #128
            # 判别网络
            #real_data = Variable(x).view(bs, -1) 
            binary_matrix, sparse_matrix = matrix_init(real_data.reshape(-1,6,9))
            fulling_matrix = SVD_func(means_filling(sparse_matrix),binary_matrix,sparse_matrix)
            fulling_matrix = fulling_matrix.reshape(-1,1,6,9).to(torch.float32)
            logits_real = D_net(real_data.reshape(-1,6,9)) # 判别网络得分
            fake_images, mu, logvar = G_net(fulling_matrix) # 生成的假的数据
            logits_fake = D_net(fake_images.reshape(-1,6,9)) # 判别网络得分

            d_total_error = discriminator_loss(logits_fake,binary_matrix) 
            d_total_error += discriminator_loss(logits_real,torch.tensor(np.ones((binary_matrix.shape[0], binary_matrix.shape[1], binary_matrix.shape[2])))) 
            D_optimizer.zero_grad()
            d_total_error.backward()
            D_optimizer.step() # 优化判别网络
            
            # 生成网络
            fake_images,mu, logvar = G_net(fulling_matrix) # 生成的假的数据
            gen_logits_fake = D_net(fake_images.reshape(-1,6,9))
            g_error = generator_loss(gen_logits_fake,binary_matrix)+ 5 * MSE_train_loss(binary_matrix, real_data.reshape(-1,6,9) , fake_images.reshape(-1,6,9)) # 生成网络的 loss
            G_optimizer.zero_grad()
            g_error.backward()
            G_optimizer.step() # 优化生成网络

            ##if (iter_count % show_every == 0):
        #if epoch%9==0:
        print('Iter: {}, D: {:.4}, G:{:.4}'.format(iter_count, d_total_error.item(), g_error.item()))
#                 imgs_numpy = deprocess_img(fake_images.data.cpu().numpy())
#                 show_images(imgs_numpy[0:16])
#                 plt.show()
#       print(MSE_train(binary_matrix,real_data.reshape(-1,6,9),fake_images))
        print(epoch)
        iter_count += 1
            

In [31]:
D = Discriminator()
G = VAE()

D_optim = get_optimizer(D)
G_optim = get_optimizer(G)

train_a_gan(D, G, D_optim, G_optim, ls_discriminator_loss, ls_generator_loss)

<ipython-input-22-bd69fe4fb5b3>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(mat)


Iter: 0, D: 0.5736, G:1.423
0
Iter: 1, D: 0.5353, G:0.8651
1
Iter: 2, D: 0.5356, G:0.5915
2
Iter: 3, D: 0.4992, G:0.5333
3
Iter: 4, D: 0.5365, G:0.4512
4
Iter: 5, D: 0.5679, G:0.3236
5
Iter: 6, D: 0.5736, G:0.2301
6
Iter: 7, D: 0.534, G:0.1838
7
Iter: 8, D: 0.5685, G:0.1794
8
Iter: 9, D: 0.5373, G:0.1827
9
Iter: 10, D: 0.551, G:0.1624
10
Iter: 11, D: 0.5471, G:0.1906
11
Iter: 12, D: 0.5363, G:0.1783
12
Iter: 13, D: 0.5346, G:0.1682
13
Iter: 14, D: 0.5634, G:0.1755
14
Iter: 15, D: 0.5903, G:0.1676
15
Iter: 16, D: 0.5356, G:0.1568
16
Iter: 17, D: 0.6011, G:0.1492
17
Iter: 18, D: 0.5956, G:0.146
18
Iter: 19, D: 0.6045, G:0.1351
19
Iter: 20, D: 0.6244, G:0.1277
20
Iter: 21, D: 0.6413, G:0.1202
21
Iter: 22, D: 0.6327, G:0.1085
22
Iter: 23, D: 0.6613, G:0.1237
23
Iter: 24, D: 0.6422, G:0.1164
24
Iter: 25, D: 0.6325, G:0.104
25
Iter: 26, D: 0.6352, G:0.1077
26
Iter: 27, D: 0.6346, G:0.105
27
Iter: 28, D: 0.636, G:0.0903
28
Iter: 29, D: 0.6482, G:0.1107
29
Iter: 30, D: 0.6543, G:0.1174
30
Iter

calculate MSE with methods of SVD-VAE-GAN and  other methods 

SVD

MICE

In [32]:
import miceforest as mf
def miceforest_test(sparse_matrix):
    sparse_matrix = sparse_matrix.to(torch.float32)
    sparse_matrix[sparse_matrix==0]=np.nan
    #sparse_matrix[:,:,7]=0
    sparse_matrix = sparse_matrix.reshape(-1,9)
    
    j = np.array(sparse_matrix)
    j = pd.DataFrame(j,columns=[str(k) for k in range(sparse_matrix.shape[1])])

    kernel = mf.MultipleImputedKernel(
        j,
        datasets=9,
        save_all_iterations=True,
        random_state=1991
    )
    kernel.mice(8)
    kernell = kernel.complete_data(8)

    ii = np.array(kernell)
    sparse_matrix = torch.tensor(ii)
    return sparse_matrix.reshape(-1,6,9)

MissForest

In [33]:
class MissForestExtra:
    def __init__(self):     
        pass
    
    def impute(self, df, impute, model, max_iter=5, verbose=0):
        df3 = df.copy()
        pd.options.mode.chained_assignment = None  # default='warn'
                  
        encode_col = [f for f in df.columns if isinstance(df[f].sample(1).values[0], str)]
        for c in encode_col:  
            df3[c].replace(df[c].dropna().unique(), range(df[c].dropna().nunique()), inplace=True) # label encoding
            if c == impute:
                impute_map = ({k:v for (k,v) in zip(df[c].dropna().unique(), range(df[c].dropna().nunique()))})
                impute_map = {v:k for (k,v) in impute_map.items()}
       
        non_null = list(set([feat for feat, val in zip(df.isnull().sum().index, df.isnull().sum()) if val == 0] + [impute]))
            
        track = {}
        for n_iter in range(max_iter):
            df2=df3[non_null].copy()
            df2['Training/Predict'] = np.where(df2[impute].isnull(), 'predict', 'training')
            predict_set = df2[df2['Training/Predict'] == 'predict']
            training_set = df2[df2['Training/Predict'] == 'training']

            if n_iter == 1:
                predict_set[impute].fillna(df2[impute].median(), inplace=True) if impute in encode_col else predict_set[impute].fillna(df2[impute].mode().values[0], inplace=True)
                   
            if n_iter > 1:
                for idx, pred in zip(predict_set.index, res):
                    predict_set.loc[idx, impute] = pred
           
            model.fit(training_set[non_null].drop(impute, axis=1), training_set[impute])
            if predict_set[non_null].drop(impute, axis=1).shape[0] != 0:
                res = model.predict(predict_set[non_null].drop(impute, axis=1))
            else:
                raise Exception(f'Feature "{impute}" does not contain any missing values.')
                
            track[f'iter{n_iter}'] = res
            
            if n_iter > 1:       
                if (track[f'iter{n_iter}'] == track[f'iter{n_iter - 1}']).all():
                    if verbose == 1:
                        print('Stopping Criteria Reached (no change in imputed values)')
                    break
            if verbose == 1:       
                if max_iter <= 1:
                    print('Stopping Criteria Reached (max iter reached)')
                    break
                   
            n_iter += 1
           
        pd.options.mode.chained_assignment = 'warn'  
       
        imputed_feature = pd.concat([training_set[impute], predict_set[impute]], axis=0).sort_index()
        return imputed_feature.replace(impute_map) if impute in encode_col else imputed_feature

In [34]:
def Missforest_test(sparse_matrix):
    from xgboost import XGBRegressor
    xgbr = XGBRegressor(use_label_encoder=False, eval_metric='merror')
    mfe = MissForestExtra()
    sparse_matrix = sparse_matrix.to(torch.float32)
    sparse_matrix[sparse_matrix==0]=np.nan
    sparse_matrix[:,:,7]=0#有一列是全0的离散序列，所以必须要返还，不然无法做XGB
    sparse_matrix = sparse_matrix.reshape(-1,9)
    
    j = np.array(sparse_matrix)
    j = pd.DataFrame(j,columns=[str(k) for k in range(sparse_matrix.shape[1])])

    for k in j.columns:
        if j[k].isnull().sum()>0:
            j[k] = mfe.impute(j, k, xgbr)
    ii = np.array(j)
    sparse_matrix = torch.tensor(ii)
    return sparse_matrix.reshape(-1,6,9)

In [35]:
def imputee(sparse_matrix):
    from sklearn.impute import SimpleImputer
    sparse_matrix = sparse_matrix.to(torch.float32)
    sparse_matrix[sparse_matrix==0]=np.nan
    sparse_matrix[:,:,7]=0#有一列是全0的离散序列，所以必须要返还，不然无法做XGB
    sparse_matrix = sparse_matrix.reshape(-1,9)
    
    j = np.array(sparse_matrix)
    j = pd.DataFrame(j,columns=[str(k) for k in range(sparse_matrix.shape[1])])

    for k in j.columns:
        if j.loc[:,k].isnull().sum()>0:
            imp_median = SimpleImputer(strategy="mean")
            j.loc[:,k] = imp_median.fit_transform(j.loc[:,k].values.reshape(-1,1))
    ii = np.array(j)
    sparse_matrix = torch.tensor(ii)
    return sparse_matrix.reshape(-1,6,9) 

In [36]:
def imputee_linear(sparse_matrix):
    from sklearn.impute import SimpleImputer
    sparse_matrix = sparse_matrix.to(torch.float32)
    sparse_matrix[sparse_matrix==0]=np.nan
    sparse_matrix[:,:,7]=0#有一列是全0的离散序列，所以必须要返还，不然无法做XGB
    sparse_matrix = sparse_matrix.reshape(-1,9)
    
    j = np.array(sparse_matrix)
    j = pd.DataFrame(j,columns=[str(k) for k in range(sparse_matrix.shape[1])])

    j=j.interpolate(method='polynomial',order=5)
    for k in j.columns:
        if j.loc[:,k].isnull().sum()>0:
            imp_median = SimpleImputer(strategy="mean")
            j.loc[:,k] = imp_median.fit_transform(j.loc[:,k].values.reshape(-1,1))
    ii = np.array(j)
    sparse_matrix = torch.tensor(ii)
    return sparse_matrix.reshape(-1,6,9) 

In [37]:
def MSE_train2(binary_matrix, origin_matrix, G_sample):
    loss = sum(sum(((1 - binary_matrix) * origin_matrix - (1 - binary_matrix) * G_sample) ** 2))/sum(sum(1 - binary_matrix))
    return loss

def test(data_iter, Label, G_net):
    torch_dataset = Data.TensorDataset(data_iter, Label)
    loader = Data.DataLoader(dataset = torch_dataset,batch_size=data.shape[0],shuffle=True)
    
    for x, (real_data,_) in enumerate(loader):
        binary_matrix, sparse_matrix = matrix_init(real_data.reshape(-1,6,9))
        fulling_matrix = SVD_func(means_filling(sparse_matrix),binary_matrix,sparse_matrix)
        fulling_matrix = fulling_matrix.reshape(-1,1,6,9).to(torch.float32)
        full_m = sparse_matrix.reshape(-1,1,6,9).to(torch.float32)
        fake_images,mu, logvar = G_net(fulling_matrix) # 生成的假的数据
        fake_images2,mu2, logvar2 = G_net(full_m)

#         print(real_data[0])
#         print(fake_images[0])
        MSEs = MSE_train(binary_matrix, real_data.reshape(-1,6,9) , fake_images.reshape(-1,6,9))
        print(MSEs)
        MSEs2 = MSE_train(binary_matrix, real_data.reshape(-1,6,9), fulling_matrix.reshape(-1,6,9))
        print(MSEs2)
        MSEs3 = MSE_train(binary_matrix, real_data.reshape(-1,6,9), miceforest_test(real_data.reshape(-1,6,9) * binary_matrix))
        print(MSEs3)
        MSEs4 = MSE_train(binary_matrix, real_data.reshape(-1,6,9), Missforest_test(real_data.reshape(-1,6,9) * binary_matrix))
        print(MSEs4)
        MSEs5 = MSE_train(binary_matrix, real_data.reshape(-1,6,9), imputee(real_data.reshape(-1,6,9) * binary_matrix))
        print(MSEs5)
        MSEs6 = MSE_train(binary_matrix, real_data.reshape(-1,6,9), fake_images2.reshape(-1,6,9))
        print(MSEs6)
        MSEs7 = MSE_train(binary_matrix, real_data.reshape(-1,6,9), imputee_linear(real_data.reshape(-1,6,9) * binary_matrix))
        print(MSEs7)

In [113]:
test(data_iter_test,Label2,G)

<ipython-input-22-bd69fe4fb5b3>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(mat)


tensor(0.0161, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.0899, dtype=torch.float64)
tensor(0.1146, dtype=torch.float64)
tensor(0.0180, dtype=torch.float64)
tensor(0.0165, dtype=torch.float64)
tensor(0.0160, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.0746, dtype=torch.float64)


用来尝试决策任务的好坏

In [71]:
def three2two(fake_images,label):
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import scale
    from sklearn.preprocessing import MinMaxScaler

    fake_images = fake_images#.to(torch.float32)
    fake_images = fake_images.detach().numpy().reshape(-1,9)
    #j=scale(X=fake_images,with_mean=True,with_std=True,copy=True)
    std=MinMaxScaler()
    j=std.fit_transform(fake_images)
    j = pd.DataFrame(j,columns=[str(k) for k in range(fake_images.shape[1])])
#     j['54']=None
#     for i in range(len(j)):
#         if label[i//6].item()>0:
#             j.loc[i,'54'] = 1
#         else:
#             j.loc[i,'54'] = 0
    j['9']=None
    for i in range(len(j)):
        if label[i//6].item()>0:
            j.loc[i,'9'] = 1
        else:
            j.loc[i,'9'] = 0
    return j    

def destree(df):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split
    from sklearn import metrics
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    from sklearn.model_selection import KFold

    
    kf = KFold(5,shuffle = True)
    
    y = df.iloc[:,-1]
    X = df.iloc[:,:-1]
    #X = df.iloc[:,:9]
    
    results=[]
    for train_index, test_index in kf.split(df):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train.astype('int'))
        y_test =  y_test.astype('int')
        y_pred =  clf.predict(X_test)
        y_pred = y_pred.astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
        auc=metrics.auc(fpr, tpr)
        precision, recall, thresholds = precision_recall_curve(y_test,y_pred, pos_label=1)
        auprc = metrics.auc(recall, precision)
        y_pred=y_pred>0.5
        acc=accuracy_score(y_test,y_pred)
        f1 = f1_score(y_test,y_pred)
        recall = recall_score(y_test,y_pred)
        result=[acc,f1,recall,auc,auprc]
        results.append(result)
    results = pd.DataFrame(results,columns = ['acc','f1','recall','auc','auprc'])
    return results  
    #return clf.score(X_test, y_test.astype('int'))
    
def Decision_tree_task(data_iter, Label, G_net):
    torch_dataset = Data.TensorDataset(data_iter, Label)
    loader = Data.DataLoader(dataset = torch_dataset,batch_size=data.shape[0],shuffle=True)
    
    for x, (real_data,label) in enumerate(loader):
        binary_matrix, sparse_matrix = matrix_init(real_data.reshape(-1,6,9))
        fulling_matrix = SVD_func(means_filling(sparse_matrix),binary_matrix,sparse_matrix)
        fulling_matrix = fulling_matrix.reshape(-1,1,6,9).to(torch.float32)
        full_m = sparse_matrix.reshape(-1,1,6,9).to(torch.float32)
        fake_images,mu, logvar = G_net(fulling_matrix) # 生成的假的数据
        fake_images2,mu2, logvar2 = G_net(full_m)
        
        print(destree(three2two(real_data.reshape(-1,6,9),label)))
        print(destree(three2two(fake_images,label)))
        print(destree(three2two(fulling_matrix.reshape(-1,6,9),label)))
        print(destree(three2two(miceforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(destree(three2two(Missforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(destree(three2two(imputee(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(destree(three2two(fake_images2.reshape(-1,6,9),label)))
        print(destree(three2two(imputee_linear(real_data.reshape(-1,6,9) * binary_matrix),label)))

In [114]:
def three2two(fake_images,label):
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import scale
    from sklearn.preprocessing import MinMaxScaler

    fake_images = fake_images#.to(torch.float32)
    fake_images = fake_images.detach().numpy().reshape(-1,9)
    #j=scale(X=fake_images,with_mean=True,with_std=True,copy=True)
    std=MinMaxScaler()
    j=std.fit_transform(fake_images)
    j = pd.DataFrame(j,columns=[str(k) for k in range(fake_images.shape[1])])
#     j['54']=None
#     for i in range(len(j)):
#         if label[i//6].item()>0:
#             j.loc[i,'54'] = 1
#         else:
#             j.loc[i,'54'] = 0
    j['9']=None
    for i in range(len(j)):
        if label[i//6].item()>0:
            j.loc[i,'9'] = 1
        else:
            j.loc[i,'9'] = 0
    return j    

def XGBdestree(df):
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    from xgboost import XGBClassifier
    from xgboost import plot_importance
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split
    from sklearn import metrics
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    from sklearn.model_selection import KFold

    
    kf = KFold(5,shuffle = True)
    
    y = df.iloc[:,-1]
    X = df.iloc[:,:-1]
    #X = df.iloc[:,:9]
    
    results=[]
    for train_index, test_index in kf.split(df):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        
        model = XGBClassifier(learning_rate=0.1,
                      #n_estimatores
                      #含义：总共迭代的次数，即决策树的个数
                      n_estimators=1000,
                      #max_depth
                      #含义：树的深度，默认值为6，典型值3-10。
                      max_depth=6,
                      #min_child_weight
                      #调参：值越大，越容易欠拟合；值越小，越容易过拟合
                      #（值较大时，避免模型学习到局部的特殊样本）。
                      min_child_weight = 1,
                      #惩罚项系数，指定节点分裂所需的最小损失函数下降值。
                      gamma = 0,
                      #subsample
                      #含义：训练每棵树时，使用的数据占全部训练集的比例。
                      # 默认值为1，典型值为0.5-1。
                      subsample = 0.8,
                      #colsample_bytree
                      #含义：训练每棵树时，使用的特征占全部特征的比例。默认值为1，典型值为0.5-1。
                      colsample_btree = 0.8,
                      #objective 目标函数
                      #multi：softmax num_class=n 返回类别
                      objective = 'multi:softmax',
                      #scale_pos_weight
                      #正样本的权重，在二分类任务中，当正负样本比例失衡时，设置正样本的权重，模型效果更好。例如，当正负样本比例为1:10时，scale_pos_weight=10
                      scale_pos_weight = 1,
                      random_state= 27
                      )
 
        model.fit(x_train,
          y_train,
          eval_set=[(x_test,y_test)],
          eval_metric = 'mlogloss',
          early_stopping_rounds = 10,
          verbose = True)
        
        y_test =  y_test.astype('int')
        y_pred =  clf.predict(X_test)
        y_pred = y_pred.astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
        auc=metrics.auc(fpr, tpr)
        precision, recall, thresholds = precision_recall_curve(y_test,y_pred, pos_label=1)
        auprc = metrics.auc(recall, precision)
        y_pred=y_pred>0.5
        acc=accuracy_score(y_test,y_pred)
        f1 = f1_score(y_test,y_pred)
        recall = recall_score(y_test,y_pred)
        result=[acc,f1,recall,auc,auprc]
        results.append(result)
    results = pd.DataFrame(results,columns = ['acc','f1','recall','auc','auprc'])
    return results  
    #return clf.score(X_test, y_test.astype('int'))
    
def XGBtask(data_iter, Label, G_net):
    torch_dataset = Data.TensorDataset(data_iter, Label)
    loader = Data.DataLoader(dataset = torch_dataset,batch_size=data.shape[0],shuffle=True)
    
    for x, (real_data,label) in enumerate(loader):
        binary_matrix, sparse_matrix = matrix_init(real_data.reshape(-1,6,9))
        fulling_matrix = SVD_func(means_filling(sparse_matrix),binary_matrix,sparse_matrix)
        fulling_matrix = fulling_matrix.reshape(-1,1,6,9).to(torch.float32)
        full_m = sparse_matrix.reshape(-1,1,6,9).to(torch.float32)
        fake_images,mu, logvar = G_net(fulling_matrix) # 生成的假的数据
        fake_images2,mu2, logvar2 = G_net(full_m)
        
        print(destree(three2two(real_data.reshape(-1,6,9),label)))
        print(destree(three2two(fake_images,label)))
        print(destree(three2two(fulling_matrix.reshape(-1,6,9),label)))
        print(destree(three2two(miceforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(destree(three2two(Missforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(destree(three2two(imputee(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(destree(three2two(fake_images2.reshape(-1,6,9),label)))
        print(destree(three2two(imputee_linear(real_data.reshape(-1,6,9) * binary_matrix),label)))

In [116]:
XGBtask(data_iter_test,Label2,G)

<ipython-input-22-bd69fe4fb5b3>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(mat)


        acc        f1    recall       auc     auprc
0  0.802778  0.336449  0.290323  0.599859  0.406272
1  0.797222  0.342342  0.372549  0.619931  0.389052
2  0.847222  0.466019  0.470588  0.689987  0.503563
3  0.800000  0.250000  0.235294  0.564249  0.305147
4  0.769444  0.394161  0.442623  0.639372  0.446165
        acc        f1    recall       auc     auprc
0  0.844444  0.508772  0.517857  0.711231  0.546429
1  0.847222  0.504505  0.538462  0.718906  0.539852
2  0.827778  0.465517  0.450000  0.676667  0.511905
3  0.825000  0.479339  0.467742  0.683535  0.525467
4  0.869444  0.515464  0.543478  0.730338  0.546004
        acc        f1    recall       auc     auprc
0  0.850000  0.578125  0.587302  0.746513  0.614377
1  0.836111  0.458716  0.446429  0.677162  0.502119
2  0.861111  0.456522  0.411765  0.673520  0.503647
3  0.863889  0.588235  0.636364  0.770641  0.619397
4  0.861111  0.553571  0.607843  0.755378  0.585798
        acc        f1    recall       auc     auprc
0  0.738889 

In [72]:
Decision_tree_task(data_iter_test,Label2,G)

<ipython-input-22-bd69fe4fb5b3>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(mat)


        acc        f1    recall       auc     auprc
0  0.794444  0.350877  0.327869  0.608750  0.409558
1  0.822222  0.384615  0.476190  0.672058  0.429941
2  0.777778  0.354839  0.343750  0.607686  0.413542
3  0.761111  0.218182  0.255319  0.546190  0.271509
4  0.811111  0.433333  0.419355  0.655986  0.483815
        acc        f1    recall       auc     auprc
0  0.863889  0.566372  0.551724  0.737783  0.602882
1  0.847222  0.455446  0.450980  0.681801  0.494379
2  0.861111  0.576271  0.596491  0.753691  0.608879
3  0.847222  0.476190  0.446429  0.683741  0.521372
4  0.855556  0.509434  0.500000  0.709150  0.547115
        acc        f1    recall       auc     auprc
0  0.786111  0.393701  0.403226  0.634499  0.445309
1  0.783333  0.338983  0.400000  0.622581  0.388725
2  0.808333  0.389381  0.415094  0.645658  0.433936
3  0.827778  0.483333  0.483333  0.690000  0.526389
4  0.783333  0.264151  0.274510  0.570912  0.315917
        acc        f1    recall       auc     auprc
0  0.822222 

In [41]:
# def init_weights(m):
#     if type(m) == nn.Linear:
#         nn.init.normal_(m.weight, std=0.01)

# def mlp_net(matrix,label):
#     #matrix = matrix.to(torch.float64)
#     #label = label.to(torch.float64)
#     net = nn.Sequential(nn.Flatten(),
#                     nn.Linear(54, 128),
#                     nn.ReLU(),
#                     nn.Linear(128, 32),
#                     nn.ReLU(),
#                     nn.Linear(32,8),
#                     nn.ReLU(),
#                     nn.Linear(8,1)
#                     )
#     net.apply(init_weights)
#     batch_size, lr, num_epochs = 300, 0.1, 150
# #     loss = nn.CrossEntropyLoss()
#     loss = nn.BCEWithLogitsLoss()
#     trainer = torch.optim.SGD(net.parameters(), lr=lr)
    
#     matrix = matrix.reshape(-1,6,9)
    
#     for epoch in range(num_epochs):
#         l = loss(net(matrix[:204]) ,label[:204].reshape(204,-1))
#         trainer.zero_grad()
#         l.backward(retain_graph=True)
#         trainer.step()
#         l = loss(net(matrix[:204]) ,label[:204].reshape(204,-1))
# #         if epoch%10==0:
# #             print(f'epoch {epoch + 1}, loss {l:f}')
#     l = loss(net(matrix[204:]) ,label[204:].reshape(96,-1))
#     print(l)
    
# def mlp_task(data_iter, Label, G_net):
#     torch_dataset = Data.TensorDataset(data_iter, Label)
#     loader = Data.DataLoader(dataset = torch_dataset,batch_size=data.shape[0],shuffle=True)
    
#     for x, (real_data,label) in enumerate(loader):
#         binary_matrix, sparse_matrix = matrix_init(real_data.reshape(-1,6,9))
#         fulling_matrix = SVD_func(means_filling(sparse_matrix),binary_matrix,sparse_matrix)
#         fulling_matrix = fulling_matrix.reshape(-1,1,6,9).to(torch.float32)
#         fake_images,mu, logvar = G_net(fulling_matrix) # 生成的假的数据
        
#         mlp_net(fake_images,label)
#         mlp_net(fulling_matrix.reshape(-1,6,9),label)
#         mlp_net(miceforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)
#         mlp_net(Missforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)
#         mlp_net(imputee(real_data.reshape(-1,6,9) * binary_matrix),label) 

In [42]:
#mlp_task(data_iter_test,Label2,G)

In [43]:
def three2two54(fake_images,label):
    fake_images = fake_images.to(torch.float32)
    fake_images = fake_images.reshape(-1,54)
    j = fake_images.detach().numpy()
    j = pd.DataFrame(j,columns=[str(k) for k in range(fake_images.shape[1])])
    j['54']=None
    for i in range(len(j)):
        if label[i].item()>0:
            j.loc[i,'54'] = 1
        else:
            j.loc[i,'54'] = 0
    return j    

def mlp_(df):
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split
    from sklearn import metrics
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    from sklearn.model_selection import KFold
    
    kf = KFold(5,shuffle = True)
    y = df.iloc[:,-1]
    X = df.iloc[:,:-1]
    print(y.sum())
    
    results=[]
    for train_index, test_index in kf.split(df):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        clf = MLPClassifier(activation='relu',max_iter=2000,
       hidden_layer_sizes=(32,16,8,4),random_state=3, shuffle=True,
       solver='adam')
        clf.fit(X_train, y_train.astype('int'))
        y_test =  y_test.astype('int')
        y_pred =  clf.predict(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
        auc=metrics.auc(fpr, tpr)
        precision, recall, thresholds = precision_recall_curve(y_test,y_pred, pos_label=1)
        auprc = metrics.auc(recall, precision)
        y_pred=y_pred>0.5
        acc=accuracy_score(y_test,y_pred)
        f1 = f1_score(y_test,y_pred)
        recall = recall_score(y_test,y_pred)
        result=[acc,f1,recall,auc,auprc]
        results.append(result)
    results = pd.DataFrame(results,columns = ['acc','f1','recall','auc','auprc'])
    print(clf.score(X_test, y_test.astype('int')))
    return results  
    #return clf.score(X_test, y_test.astype('int'))
    
def mlpp_task(data_iter, Label, G_net):
    torch_dataset = Data.TensorDataset(data_iter, Label)
    loader = Data.DataLoader(dataset = torch_dataset,batch_size=data.shape[0],shuffle=True)
    
    for x, (real_data,label) in enumerate(loader):
        binary_matrix, sparse_matrix = matrix_init(real_data.reshape(-1,6,9))
        fulling_matrix = SVD_func(means_filling(sparse_matrix),binary_matrix,sparse_matrix)
        fulling_matrix = fulling_matrix.reshape(-1,1,6,9).to(torch.float32)
        fake_images,mu, logvar = G_net(fulling_matrix) # 生成的假的数据
        
        print(fake_images.shape)
        print(mlp_(three2two54(fake_images,label)))
        print(mlp_(three2two54(fulling_matrix.reshape(-1,6,9),label)))
        print(mlp_(three2two54(miceforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(mlp_(three2two54(Missforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(mlp_(three2two54(imputee(real_data.reshape(-1,6,9) * binary_matrix),label)))

In [44]:
mlpp_task(data_iter_test,Label2,G)

<ipython-input-22-bd69fe4fb5b3>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(mat)


torch.Size([300, 1, 6, 9])
46
0.8666666666666667
        acc   f1  recall  auc     auprc
0  0.883333  0.0     0.0  0.5  0.558333
1  0.816667  0.0     0.0  0.5  0.591667
2  0.883333  0.0     0.0  0.5  0.558333
3  0.783333  0.0     0.0  0.5  0.608333
4  0.866667  0.0     0.0  0.5  0.566667
46
0.7
        acc        f1    recall       auc     auprc
0  0.866667  0.333333  0.250000  0.605769  0.425000
1  0.833333  0.166667  0.111111  0.535948  0.288889
2  0.816667  0.266667  0.200000  0.570000  0.366667
3  0.816667  0.266667  0.250000  0.576923  0.317857
4  0.700000  0.100000  0.090909  0.463822  0.184343
46
0.8
        acc   f1  recall       auc     auprc
0  0.916667  0.0     0.0  0.500000  0.541667
1  0.883333  0.0     0.0  0.490741  0.050000
2  0.816667  0.0     0.0  0.500000  0.591667
3  0.800000  0.0     0.0  0.500000  0.600000
4  0.800000  0.0     0.0  0.500000  0.600000
46
0.8166666666666667
        acc        f1    recall       auc     auprc
0  0.800000  0.250000  0.166667  0.562500

In [45]:
def three2two54(fake_images,label):
    fake_images = fake_images.to(torch.float32)
    fake_images = fake_images.reshape(-1,54)
    j = fake_images.detach().numpy()
    print(j)
    j = pd.DataFrame(j,columns=[str(k) for k in range(fake_images.shape[1])])
    j['54']=None
    for i in range(len(j)):
        if label[i].item()>0:
            j.loc[i,'54'] = 1
        else:
            j.loc[i,'54'] = 0
    return j    

def svm_(df):
    from sklearn.svm import LinearSVC
    from sklearn.model_selection import train_test_split
    from sklearn import metrics
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    from sklearn.preprocessing import label_binarize
    from sklearn.model_selection import KFold
    
    kf = KFold(5,shuffle = True)
    
    y = df.iloc[:,-1]
    X = df.iloc[:,:-1]
    
    print(df)
    results=[]
    for train_index, test_index in kf.split(df):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        print(X_train.shape,X_test.shape)
        clf = LinearSVC(penalty='l2', loss='squared_hinge',random_state=10)
        clf.fit(X_train, y_train.astype('int'))
        y_test =  y_test.astype('int')
        y_pred =  clf.predict(X_test)
        y_test = np.array(y_test).reshape(y_pred.shape)
        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
        auc=metrics.auc(fpr, tpr)
        precision, recall, thresholds = precision_recall_curve(y_test,y_pred, pos_label=1)
        auprc = metrics.auc(recall, precision)
        #y_pred=y_pred>0.5
        acc=accuracy_score(y_test,y_pred)
        f1 = f1_score(y_test,y_pred)
        recall = recall_score(y_test,y_pred)
        result=[acc,f1,recall,auc,auprc]
        results.append(result)
    results = pd.DataFrame(results,columns = ['acc','f1','recall','auc','auprc'])
    print(clf.score(X_test, y_test.astype('int')))
    return results  
    #return clf.score(X_test, y_test.astype('int'))
    
def svm_task(data_iter, Label, G_net):
    torch_dataset = Data.TensorDataset(data_iter_train, Label)
    loader = Data.DataLoader(dataset = torch_dataset,batch_size=data.shape[0],shuffle=True)
    
    for x, (real_data,label) in enumerate(loader):
        binary_matrix, sparse_matrix = matrix_init(real_data.reshape(-1,6,9))
        fulling_matrix = SVD_func(means_filling(sparse_matrix),binary_matrix,sparse_matrix)
        fulling_matrix = fulling_matrix.reshape(-1,1,6,9).to(torch.float32)
        fake_images,mu, logvar = G_net(fulling_matrix) # 生成的假的数据
        
        print(svm_(three2two54(fake_images,label)))
        print(svm_(three2two54(fulling_matrix.reshape(-1,6,9),label)))
        print(svm_(three2two54(miceforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(svm_(three2two54(Missforest_test(real_data.reshape(-1,6,9) * binary_matrix),label)))
        print(svm_(three2two54(imputee(real_data.reshape(-1,6,9) * binary_matrix),label)))

In [46]:
svm_task(data_iter_test,Label,G)

AssertionError: Size mismatch between tensors